In [1]:
#get IBD rank
#http://myibd.investors.com/search/searchresults.aspx?Ntt=thor
#<a href="http://myibd.investors.com/search/searchresults.aspx?Ntt=Thor Industries Inc&amp;source=manager&amp;in_dym=true" id="ctl00_ctl00_secondaryContent_leftContent_SearchResultsMgrCtrl_didYouMeanCompanyRepeater_ctl00_didYouMeanLink">Thor Industries Inc</a>
#1st: <span id="ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_ltlSymbolRank">1</span>
#not 1st: <span id="ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_ltlSymbolRank">7</span>
#<a class="stockRoll" href="http://research.investors.com/stock-quotes/nasdaq-alarmcom-holdings-inc-alrm.htm" rel="/stocksymbol.axd?symbol=ALRM">ALRM</a>
#IBD group:
#<div class="spansubHead">Bldg-Mobile/Mfg &amp; Rv Group</div>
#<a class="stockRoll" href="http://research.investors.com/stock-quotes/nasdaq-alarmcom-holdings-inc-alrm.htm" rel="/stocksymbol.axd?symbol=ALRM">ALRM</a>
#<span id="ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_ltlSymbolRank">1</span>

In [2]:
import requests
from bs4 import BeautifulSoup as BSoup

In [14]:
#download the page
r = requests.get("http://myibd.investors.com/search/searchresults.aspx?Ntt=aeis")
#now the html code is in the requests object:
r.url

'http://myibd.investors.com/search/searchresults.aspx?Ntt=aeis'

In [15]:
#BeautifulSoup can parse html code and convert into something usable
soup =  BSoup (r.content, "lxml")
#e.g. print html code in readable format
#print (soup.prettify())

In [16]:
#now can find all <a>-tags
#soup.find_all ("a")
#get all the hyperlinks
#for link in soup.find_all("a"):
#    print (link.get ("href"))
    #the link text
    #link.text
    #<span id="ctl00_ctl00_secondaryContent_leftContent_CustomContentCtrl_StockTwitMiniChart1_lblCompany" onclick="javascript:window.open('http://research.investors.com/stock-quotes/nasdaq-atlassian-corp-plc-cl-a-team.htm')">Atlassian Corp Plc Cl A</span>

In [17]:
#can find the link to the stock's page
stock_page = soup.find_all ("a", {"id": "ctl00_ctl00_secondaryContent_leftContent_SearchResultsMgrCtrl_didYouMeanCompanyRepeater_ctl00_didYouMeanLink"})
if not stock_page:
    print (stock_page)
for link in stock_page:
    print (link.get ("href"), link.text)

[]


In [18]:
#help(stock_page[0])

In [19]:
#print (stock_page[0], "\n")
#print (stock_page[0].get_text(), "\n")
#print (stock_page[0].get('href'))

In [20]:
#follow down the link
#r2=requests.get(stock_page[0].get('href'))

In [22]:
soup2 = BSoup (r.content, "lxml")
#<span id="ctl00_ctl00_secondaryContent_leftContent_CustomContentCtrl_StockTwitMiniChart1_lblCompany" onclick="javascript:window.open('http://research.investors.com/stock-quotes/nyse-thor-industries-inc-tho.htm')">Thor Industries Inc</span>
stock_page2 = soup2.find ("span", {"id": "ctl00_ctl00_secondaryContent_leftContent_CustomContentCtrl_StockTwitMiniChart1_lblCompany"})
print (stock_page2.get ('onclick'))
tokens = stock_page2.get ('onclick').split ("'")
for t in tokens:
    if "http://" in t:
        stock_link = t
        break
r3 = requests.get (stock_link)
soup3 = BSoup (r3.content, "lxml")
#can find all <span>'s with a certain id:

#spans_id1 = soup3.find_all ("span")
#there is only one occurrence of this (should verify?)
spans_id1 = soup3.find ("span", {"id":"ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_ltlSymbolRank"})
print (spans_id1.text)

#current rank:
#<div id="ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_currentRank" class="divSymbolRank"><span class="rank-left">#</span><span id="ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_spanSymbolRank"><span id="ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_ltlSymbolRank">4</span></span></div>

#first rank:
group_no1 = soup3.find ("a", {"class": "stockRoll"})
#print (len(group_no1))
print (group_no1)

market_group = soup3.find ("div", {"class": "spansubHead"})
print (market_group.text)


javascript:window.open('http://research.investors.com/stock-quotes/nasdaq-advanced-energy-inds-aeis.htm')
1
None
Elec-Semiconductor Equip Group


In [5]:
import pandas as pd
df = pd.read_excel ("screener_results.xls")
print (df.columns)
#Symbol's are in alphabetical order, should eliminate everything past the first 'NaN'
for s in df.Symbol:
    print (df["Company Name"][df.Symbol == s])

Index(['Symbol', 'Company Name', 'Security Type', 'Security Price',
       'EPS Growth (Last Qtr vs. Same Qtr Prior Yr)',
       'EPS Growth (TTM vs Prior TTM)', 'Return on Equity (TTM)',
       'Positive Earnings Surprises (90 Days)',
       'Institutional Ownership (Last vs. Prior Qtr)',
       'Net Insider Shares Bought', 'Sector', 'Industry', 'Sub-Industry'],
      dtype='object')
0    Adobe Systems Inc
Name: Company Name, dtype: object
1    Advanced Energy Industries Inc.
Name: Company Name, dtype: object
2    Alibaba Group Holding Ltd
Name: Company Name, dtype: object
3    Align Technology Inc
Name: Company Name, dtype: object
4    Alliance Data Systems Corp
Name: Company Name, dtype: object
5    Allison Transmission Holdings Inc
Name: Company Name, dtype: object
6    Ameriprise Financial Inc
Name: Company Name, dtype: object
7    Antero Midstream Partners LP
Name: Company Name, dtype: object
8    Applied Industrial Technologies Inc
Name: Company Name, dtype: object
9    Applied 

136    Sealed Air Corp
Name: Company Name, dtype: object
137    Service Corp International
Name: Company Name, dtype: object
138    ServiceMaster Global Holdings Inc
Name: Company Name, dtype: object
139    SiteOne Landscape Supply Inc
Name: Company Name, dtype: object
140    Skyworks Solutions Inc
Name: Company Name, dtype: object
141    Stamps.com Inc
Name: Company Name, dtype: object
142    State National Companies Inc
Name: Company Name, dtype: object
143    Supernus Pharmaceuticals Inc
Name: Company Name, dtype: object
144    Sysco Corp
Name: Company Name, dtype: object
145    T. Rowe Price Group Inc
Name: Company Name, dtype: object
146    TAL Education Group
Name: Company Name, dtype: object
147    Telecom Argentina SA
Name: Company Name, dtype: object
148    Tempur Sealy International Inc
Name: Company Name, dtype: object
149    Ternium SA
Name: Company Name, dtype: object
150    Texas Instruments Inc
Name: Company Name, dtype: object
151    Texas Pacific Land Trust
Name: Compa